In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import librosa
import torch
import penne

In [ ]:
loader = penne.data.loader(['mdb'], 'test')
penne.PERIODICITY = 'sum'
penne.DECODER = 'viterbi'
penne.METHOD = 'pyin'

In [ ]:
metrics = penne.evaluate.Metrics()

for audio, bins, pitch, voiced, stem in loader:
    metrics.reset()

    # Pad
    pad = (penne.WINDOW_SIZE - penne.HOPSIZE) // 2
    audio = torch.nn.functional.pad(audio, (pad, pad))

    # Infer
    logits = penne.dsp.pyin.infer(audio[0])

    # Update metrics
    args = logits, bins, pitch, voiced
    metrics.update(*args)

    lib_pitch, _, lib_period = librosa.pyin(
        audio.numpy(),
        fmin=penne.FMIN,
        fmax=penne.FMAX,
        sr=penne.SAMPLE_RATE,
        win_length=penne.WINDOW_SIZE,
        hop_length=penne.HOPSIZE,
        center=False)
        
    break

In [ ]:
pred_bins, pred_pitch, pred_period = penne.postprocess(logits)

In [ ]:
pred_bins.shape, pred_pitch.shape

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(18, 8))
plt.plot(pitch.squeeze()[voiced.squeeze()])
# plt.plot(lib_pitch.squeeze()[voiced.squeeze()])
plt.plot(pred_pitch.squeeze()[voiced.squeeze()])



In [ ]:
plt.figure(figsize=(18, 8))
plt.plot(pred_period.squeeze())
plt.plot(lib_period.squeeze())

In [ ]:
metrics()